In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import torch
import os, random

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# fix seed
seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load data
path = '/content/drive/MyDrive/Colab Notebooks/JPX_Fundamental/data/'
stock_list = pd.read_csv(os.path.join(path, 'stock_list.csv'))
stock_price = pd.read_csv(os.path.join(path, 'stock_price.csv'))
stock_fin = pd.read_csv(os.path.join(path, 'stock_fin.csv'), index_col='base_date')
stock_labels = pd.read_csv(os.path.join(path, 'stock_labels.csv'))

# Datetimeに変換
stock_price['EndOfDayQuote Date'] = pd.to_datetime(stock_price['EndOfDayQuote Date'])
stock_fin.index = pd.to_datetime(stock_fin.index)

In [ ]:
# load
df_all_code_tech = pd.read_csv(os.path.join(path, 'all_code_tech.csv'))
df_all_code_tech.set_index('Unnamed: 0', drop=True, inplace=True) # こうしないとエラーが出る
df_all_code_fund = pd.read_csv(os.path.join(path, 'all_code_fund.csv'), index_col=[0])

In [ ]:
# 目的変数
# high-lowの差分
stock_labels['high_low_5'] = stock_labels['label_high_5'] - stock_labels['label_low_5']
stock_labels['high_low_10'] = stock_labels['label_high_10'] - stock_labels['label_low_10']
stock_labels['high_low_20'] = stock_labels['label_high_20'] - stock_labels['label_low_20']

# high-lowの中間
stock_labels['center_5'] = (stock_labels['label_high_5'] + stock_labels['label_low_5']) / 2
stock_labels['center_10'] = (stock_labels['label_high_10'] + stock_labels['label_low_10']) / 2
stock_labels['center_20'] = (stock_labels['label_high_20'] + stock_labels['label_low_20']) / 2

stock_labels.head()

,base_date,Local Code,label_date_5,label_high_5,label_low_5,label_date_10,label_high_10,label_low_10,label_date_20,label_high_20,label_low_20,high_low_5,high_low_10,high_low_20,center_5,center_10,center_20
0,2016-01-04,1301,2016-01-12,0.01091,-0.04000,2016-01-19,0.01091,-0.05455,2016-02-02,0.01091,-0.08727,0.05091,0.06546,0.09818,-0.014545,-0.021820,-0.03818
1,2016-01-05,1301,2016-01-13,0.00362,-0.04348,2016-01-20,0.00362,-0.07609,2016-02-03,0.00362,-0.09058,0.04710,0.07971,0.09420,-0.019930,-0.036235,-0.04348
2,2016-01-06,1301,2016-01-14,0.00000,-0.05072,2016-01-21,0.00000,-0.08696,2016-02-04,0.00362,-0.09058,0.05072,0.08696,0.09420,-0.025360,-0.043480,-0.04348
3,2016-01-07,1301,2016-01-15,0.01107,-0.03321,2016-01-22,0.01107,-0.07380,2016-02-05,0.02214,-0.07380,0.04428,0.08487,0.09594,-0.011070,-0.031365,-0.02583
4,2016-01-08,1301,2016-01-18,0.01111,-0.03333,2016-01-25,0.01111,-0.07037,2016-02-08,0.02593,-0.07037,0.04444,0.08148,0.09630,-0.011110,-0.029630,-0.02222


# Dataset Class

In [ ]:
from torch.utils.data import Dataset
from torch import nn

class JPX_Dataset(Dataset):
    
    def __init__(self, dfs, window_size):
        self.technical_index = dfs['technical_index']
        self.fundamental_index = dfs['fundamental_index']
        self.stock_label = dfs['stock_label']
        self.window_size = window_size
        
        # fundamental_indexの日付から直近windows_size日遡れないデータは削除する
        self.arrange_fund_table()
    
    def arrange_fund_table(self):
        '直近windows_size日分、遡れないデータは削除する'
        del_index = []
        
        # 削除対象インデックスを特定
        for i, code in enumerate(sorted(set(self.fundamental_index['Local Code']))):
            # 1銘柄に関するデータ
            df_one_code_tech = self.technical_index.loc[self.technical_index['Local Code'] == code].copy()
            df_one_code_fund = self.fundamental_index.loc[self.fundamental_index['Local Code'] == code].copy()

            # 時系列データの最初の日付を取得
            first_date = df_one_code_tech['EndOfDayQuote Date'].values[0]

            check = True
            for j in df_one_code_fund.index:
                if not check:
                    break

                if df_one_code_fund['base_date'][j] < first_date:
                    del_index.append(j)
                    continue
                else:
                    # ファンダメンタルデータと時系列データの日付同じインデックス番号を取得
                    same_date_index = df_one_code_tech[df_one_code_tech['EndOfDayQuote Date'] == df_one_code_fund['base_date'][j]].index[0]
                    # 判定
                    if df_one_code_tech.index[0] > same_date_index - self.window_size:
                        del_index.append(j)
                    else:
                        check = False
            
        self.fundamental_index = self.fundamental_index.drop(del_index, axis=0).copy()
        self.fundamental_index = self.fundamental_index.reset_index(drop=True)
        
    def get_past_data(self, code, base_date, n):
        '''銘柄(code)の基準日からn日前～基準日のデータを取り出す'''
        # 1銘柄に関する価格情報を取り出す
        df_one_stock = self.technical_index[self.technical_index['Local Code'] == code].copy()
        
        # 過去データを取り出す
        extract_cols = ['EndOfDayQuote Date', 'EndOfDayQuote Volume', 'log_R', 'return_5', 'return_25', 'return_75', 
                        'HV_5', 'HV_10', 'HV_25', "HV_50", 'HV_75', 'HV_100',
                        'MA20_HV5', 'MA20_HV10', 'MA20_HV25', 'MA20_HV50', 'MA20_HV75', 'MA20_HV100', 
                        'MADR5', 'MADR25', 'MADR75', 'MXDR5', 'MXDR10', 'MXDR20', 'MNDR5', 'MNDR10', 'MNDR20', 'RNDR', 
                        'RSI', 'H-L_C', 'MA25_H-L_C']
        
        base_date_index = df_one_stock[df_one_stock['EndOfDayQuote Date'] == base_date].index[0]
        
        return df_one_stock.loc[base_date_index-n+1:base_date_index][extract_cols].iloc[:, 1:].values
    
    def __len__(self):
        return len(self.fundamental_index)
     
    def __getitem__(self, i):
        # インデックスiに対応する銘柄コード、日付を取り出す
        code_i = self.fundamental_index['Local Code'][i]
        date_i = self.fundamental_index['base_date'][i]
        #print(code_i, date_i)
        
        # 過去の時系列データ(windowsize_分)を取り出す
        data_ts = self.get_past_data(code_i, date_i, self.window_size)
        data_cs = self.fundamental_index.iloc[i, 2:].values.astype(np.float64)
        
        # ラベルデータを取り出す
        stock_label_target = self.stock_label[(self.stock_label['base_date']== date_i) & (self.stock_label['Local Code']==code_i)]
        label = stock_label_target[['label_high_20', 'label_low_20', 'high_low_20', 'center_20']].values
        
        # numpy -> Torch,Tensor
        data_ts = torch.from_numpy(data_ts).float()
        data_cs = torch.from_numpy(data_cs).float()
        label = torch.from_numpy(label).float()
        
        return data_ts, data_cs, label

In [ ]:
# 前処理
df_all_code_tech['EndOfDayQuote Date'] = pd.to_datetime(df_all_code_tech['EndOfDayQuote Date'])
df_all_code_fund['base_date'] = pd.to_datetime(df_all_code_fund['base_date'])
stock_labels['base_date'] = pd.to_datetime(stock_labels['base_date'])

delete_cols = df_all_code_fund.iloc[:, 2:43].columns
df_all_code_fund.fillna(0, inplace=True)
df_all_code_fund.reset_index(drop=True, inplace=True)


# 辞書にまとめる
dfs = {
    'technical_index':df_all_code_tech,
    'fundamental_index':df_all_code_fund.drop(delete_cols, axis=1),
    'stock_label':stock_labels,
}

window_size = 5
ds = JPX_Dataset(dfs, window_size)

In [ ]:
ds[20][0]

tensor([[ 4.7543e+06,  4.0694e-02,  1.0526e-01,  0.0000e+00,  0.0000e+00,
          1.1180e-02,  4.4489e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  7.0724e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00, -7.4753e-02,  8.3189e-02,  8.5069e-02,
          8.5069e-02, -7.0000e-02,  5.6311e-01,  4.7619e-02,  0.0000e+00],
        [ 4.9409e+06, -4.6118e-03,  1.1149e-01,  0.0000e+00,  0.0000e+00,
          2.8243e-02,  4.7180e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  4.3478e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  1.1438e-01,  1.1632e-01,
          1.1632e-01,  8.0000e-02,  5.0794e-01,  4.1667e-02,  0.0000e+00],
        [ 4.1135e+06, -3.1301e-02,  7.5342e-02,  0.0000e+00,  0.0000e+00,
          2.9787e-02,  4.7774e-02,  

In [ ]:
ds.technical_index[ds.technical_index['Local Code'] == 1332].loc[1240:].head()

,Local Code,EndOfDayQuote Date,EndOfDayQuote Open,EndOfDayQuote High,EndOfDayQuote Low,EndOfDayQuote Close,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote Volume,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote PreviousClose,EndOfDayQuote PreviousCloseDate,EndOfDayQuote PreviousExchangeOfficialClose,EndOfDayQuote PreviousExchangeOfficialCloseDate,EndOfDayQuote ChangeFromPreviousClose,EndOfDayQuote PercentChangeFromPreviousClose,EndOfDayQuote VWAP,log_R,return_5,return_25,return_75,HV_5,HV_10,HV_25,HV_50,HV_75,HV_100,MA20_HV5,MA20_HV10,MA20_HV25,MA20_HV50,MA20_HV75,MA20_HV100,MADR5,MADR25,MADR75,MXDR5,MXDR10,MXDR20,MNDR5,MNDR10,MNDR20,RNDR,RSI,H-L_C,MA5_H-L_C,MA10_H-L_C,MA25_H-L_C,MA50_H-L_C,MA75_H-L_C,MA100_H-L_C
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1240,1332,2016-02-01,630.0,656.0,625.0,651.0,651.0,4754300.0,1.0,625.0,2016/01/29,625.0,2016/01/29,26.0,4.160,644.696,0.040694,0.105263,0.0,0.0,0.011180,0.044489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.070724,0.0,0.0,0.000000,0.000000,-0.074753,0.083189,0.085069,0.085069,-0.070000,0.563107,0.047619,0.040782,0.044310,0.0,0.0,0.0,0.0
1241,1332,2016-02-02,646.0,670.0,643.0,648.0,648.0,4940900.0,1.0,651.0,2016/02/01,651.0,2016/02/01,-3.0,-0.461,657.737,-0.004612,0.111492,0.0,0.0,0.028243,0.047180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.000000,0.000000,0.000000,0.114385,0.116319,0.116319,0.080000,0.507937,0.041667,0.043627,0.045997,0.0,0.0,0.0,0.0
1242,1332,2016-02-03,636.0,637.0,614.0,628.0,628.0,4113500.0,1.0,648.0,2016/02/02,648.0,2016/02/02,-20.0,-3.086,625.736,-0.031301,0.075342,0.0,0.0,0.029787,0.047774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002858,0.0,0.0,-0.049254,-0.049254,-0.049254,0.062284,0.065972,0.065972,0.046667,0.533333,0.036624,0.044102,0.045039,0.0,0.0,0.0,0.0
1243,1332,2016-02-04,621.0,625.0,592.0,596.0,596.0,3378300.0,1.0,628.0,2016/02/03,628.0,2016/02/03,-32.0,-5.096,603.018,-0.052214,-0.001675,0.0,0.0,0.025945,0.045369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.053367,0.0,0.0,-0.067164,-0.067164,-0.067164,0.000000,0.025997,0.027778,-0.006667,0.461538,0.055369,0.047136,0.044857,0.0,0.0,0.0,0.0
1244,1332,2016-02-05,590.0,624.0,580.0,591.0,591.0,6473000.0,1.0,596.0,2016/02/04,596.0,2016/02/04,-5.0,-0.839,594.383,-0.008410,-0.054400,0.0,0.0,0.033763,0.035279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.051060,0.0,0.0,-0.068657,-0.068657,-0.068657,0.000000,0.005199,0.006944,-0.015000,0.454976,0.074450,0.051146,0.048163,0.0,0.0,0.0,0.0


In [ ]:
ds.fundamental_index[ds.fundamental_index['Local Code'] == 1332].head()

,base_date,Local Code,OperatingIncome_NetSales,OrdinaryIncome_NetSales,NetIncome_NetSales,NetSales_Growth,OperatingIncome_Growth,OrdinaryIncome_Growth,NetIncome_Growth,Forecast_NetSales_Growth,Forecast_OperatingIncome_Growth,Forecast_OrdinaryIncome_Growth,Forecast_NetIncome_Growth,Capital_Ratio,ROE,ROA,EndOfDayQuote ExchangeOfficialClose,Dividend_Yeild,CF_Operating_pn_-1.0,CF_Operating_pn_0.0,CF_Operating_pn_1.0,CF_Financing_pn_-1.0,CF_Financing_pn_0.0,CF_Financing_pn_1.0,CF_Investing_pn_-1.0,CF_Investing_pn_0.0,CF_Investing_pn_1.0,17_Sector_1,17_Sector_2,17_Sector_3,17_Sector_4,17_Sector_5,17_Sector_6,17_Sector_7,17_Sector_8,17_Sector_9,17_Sector_10,17_Sector_11,17_Sector_12,17_Sector_13,17_Sector_14,17_Sector_15,17_Sector_16,17_Sector_17,Result_FinancialStatement ReportType_Annual,Result_FinancialStatement ReportType_Q1,Result_FinancialStatement ReportType_Q2,Result_FinancialStatement ReportType_Q3
20,2016-02-05,1332,0.035628,0.038829,0.023085,0.000000,0.000000,0.000000,0.000000,0.314927,0.038002,0.058257,0.023496,0.240797,0.098131,0.023630,591.0,0.000000,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21,2016-05-13,1332,0.030513,0.032481,0.019315,0.000000,0.000000,0.000000,0.000000,-0.505622,-0.588520,-0.589293,-0.634354,0.255841,0.107928,0.027612,552.0,0.005435,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
22,2016-08-05,1332,0.026698,0.025061,0.010579,0.000000,0.000000,0.000000,0.000000,1.014827,0.916627,1.169474,1.720677,0.246581,0.015293,0.003771,453.0,0.006623,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
23,2016-11-04,1332,0.032302,0.030406,0.015450,0.000000,0.000000,0.000000,0.000000,0.991704,0.987362,1.165440,1.557000,0.276056,0.038613,0.010659,522.0,0.004789,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
24,2017-02-21,1332,0.039917,0.044196,0.026400,-0.036773,0.079177,0.096354,0.101549,0.339530,0.148873,0.134170,0.131130,0.291469,0.093327,0.027202,562.0,0.004448,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# Train/Validation Dataset

In [ ]:
# データの分割期間の設定
TRAIN_END = "2017-11-30"
VAL_START = "2018-01-01"
VAL_END = "2018-12-01"
TEST_START = "2019-01-01"

In [ ]:
# 各データのインデックスを取得
train_index = ds.fundamental_index.index[ds.fundamental_index['base_date'] <= TRAIN_END].values
val_index = ds.fundamental_index.index[(ds.fundamental_index['base_date'] >= VAL_START) & (ds.fundamental_index['base_date'] <= VAL_END)] .values
test_index = ds.fundamental_index.index[ds.fundamental_index['base_date'] >= TEST_START].values

In [ ]:
from torch.utils.data.dataset import Subset

train_ds = Subset(ds, train_index)
valid_ds = Subset(ds, val_index)
test_ds = Subset(ds, test_index)


print('train size:',len(train_ds))
print('valid size:',len(valid_ds))
print('test size:',len(test_ds))

train size: 26509
valid size: 13662
test size: 29027


# DataLoader

In [ ]:
batch_size = 2048

# make DataLoder
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=True)

# dict
dataloaders_dict = {'train': train_dataloader,
                    'val'  : valid_dataloader,
                    'test' : test_dataloader}

In [ ]:
# Check
batch_iterator = iter(dataloaders_dict['train'])
inputs_ts, inputs_cs, labels = next(batch_iterator)
print(inputs_ts.size(), inputs_cs.size(), labels.size())

torch.Size([2048, 5, 30]) torch.Size([2048, 46]) torch.Size([2048, 1, 4])


## モデル構築
Network(TCN)  
https://github.com/locuslab/TCN/blob/master/TCN/tcn.py

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU() #nn.SiLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        
        self.fc1 = nn.Linear(106, 128)
        self.dropout1 = nn.Dropout(dropout)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.LeakyReLU1 = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        #self.SiLU1 = nn.SiLU()
        
        self.fc2 = nn.Linear(128,128)
        self.dropout2 = nn.Dropout(dropout)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.LeakyReLU2 = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        #self.SiLU2 = nn.SiLU()
        
        self.fc3 = nn.Linear(128, output_size)
        
    def forward(self, inputs_ts, inputs_cs):
        
        y1 = self.tcn(inputs_ts)  # input should have dimension (N, C, L)
        y1 = torch.flatten(y1, start_dim=1)
        
        # concate:y1(N, 60) + inputs_cs(N, 46) -> (N, 106)
        y1 = torch.cat([y1, inputs_cs], dim=1)
        
        y1 = self.fc1(y1)
        y1 = self.batch_norm1(y1)
        y1 = self.LeakyReLU1(y1)
        #y1 = self.SiLU1(y1)
        
        y1 = self.dropout1(y1)
        
        y1 = self.fc2(y1)
        y1 = self.batch_norm2(y1)
        y1 = self.LeakyReLU2(y1)
        #y1 = self.SiLU2(y1)
        
        y1 = self.dropout2(y1)
        
        o = self.fc3(y1)
        return o #torch.sigmoid(o)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('use devise:', device)

net = TCN(input_size=window_size, output_size=4, num_channels=[16, 8, 4, 2], kernel_size=2, dropout=0.5)
print(net)

use devise: cuda:0
TCN(
  (gnoise): GaussianNoise()
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(5, 16, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.5, inplace=False)
        (conv2): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.5, inplace=False)
        (net): Sequential(
          (0): Conv1d(5, 16, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.5, inplace=False)
          (4): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(1,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.5, inplace=False)
        )
        (downsample): Conv1d(5, 16, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): C

In [ ]:
inputs_ts.size()

torch.Size([2048, 5, 30])

In [ ]:
# Check
#o = net(inputs_ts, inputs_cs)
#print(o.size())
#print(train_ds[0][2].size())
#o

In [ ]:
#o.unsqueeze(1).size()

# Loss Function/Optim

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train

In [ ]:
# load model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
net.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/JPX_Fundamental/models/DL/model.mdl'))

cuda:0


<All keys matched successfully>

In [ ]:
from tqdm import tqdm

def train_model(net, dataloader_dict, criterion, optimizer, num_epochs):
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('use devise:', device)
    
    net.to(device)
    #torch.backends.cudnn.deterministic = True
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('--------------------------')
        
        for phase in ['train', 'val']:

            epoch_loss = 0.0
        
            for inputs_ts, inputs_cs, labels in tqdm(dataloader_dict[phase]):
                
                inputs_ts = inputs_ts.to(device)
                inputs_cs = inputs_cs.to(device)
                labels = labels.to(device)

                # init optimizer:勾配パラメータを0にする
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = net(inputs_ts, inputs_cs)
                    outputs = outputs.unsqueeze(1) #(N, 4) -> (N, 1, 4)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    epoch_loss += loss.item() * inputs_ts.size(0)
                
            # print Score
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            
        # save model
        if phase == 'val':
            if epoch == 0:
                best_val_loss = epoch_loss
                save = True
            elif best_val_loss > epoch_loss:
                best_val_loss = epoch_loss
                save = True
            if save:
                print('Best score updated. New model was saved.')
                torch.save(net.state_dict(), '/content/drive/MyDrive/Colab Notebooks/JPX_Fundamental/models/DL/model.mdl' )
                save = False

In [ ]:
num_epochs = 200
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

use devise: cuda:0


  0%|          | 0/13 [00:00<?, ?it/s]

Epoch 1/200
--------------------------


  0%|          | 0/7 [00:00<?, ?it/s]

train Loss: 5847.4283


...

  0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 345.1088
Best score updated. New model was saved.
Epoch 12/200
--------------------------


  0%|          | 0/7 [00:00<?, ?it/s]

train Loss: 799.3490


 86%|████████▌ | 6/7 [04:48<00:48, 48.11s/it]

In [ ]:
num_epochs = 200
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

  0%|          | 0/13 [00:00<?, ?it/s]

use devise: cuda:0
Epoch 1/200
--------------------------


  0%|          | 0/7 [00:00<?, ?it/s]

train Loss: 745.4716


  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

train Loss: 474.1574


  0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 173.9870
Best score updated. New model was saved.
Epoch 14/200
--------------------------


 85%|████████▍ | 11/13 [08:49<01:36, 48.16s/it]

In [ ]:
# load model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
net.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/JPX_Fundamental/models/DL/model.mdl'))

num_epochs = 200
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

  0%|          | 0/13 [00:00<?, ?it/s]

cuda:0
use devise: cuda:0
Epoch 1/200
--------------------------


  0%|          | 0/7 [00:00<?, ?it/s]

train Loss: 474.1308


  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

train Loss: 468.0246


  0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 172.2068
Best score updated. New model was saved.
Epoch 5/200
--------------------------


 62%|██████▏   | 8/13 [06:31<04:04, 48.99s/it]

In [ ]:
# load model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
net.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/JPX_Fundamental/models/DL/model.mdl'))

num_epochs = 200
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

  0%|          | 0/13 [00:00<?, ?it/s]

cuda:0
use devise: cuda:0
Epoch 1/200
--------------------------


  0%|          | 0/7 [00:00<?, ?it/s]

train Loss: 458.1860


  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 168.7657
Best score updated. New model was saved.
Epoch 15/200
--------------------------


  0%|          | 0/7 [00:00<?, ?it/s]

train Loss: 465.5570


 14%|█▍        | 1/7 [00:49<04:56, 49.45s/it]